In [1]:
! nvidia-smi

Mon Nov 30 14:50:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.33.01    Driver Version: 440.33.01    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    35W /  70W |     11MiB / 15109MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Tesla T4            Off  | 00000000:00:05.0 Off |                    0 |
| N/A   77C    P0    66W /  70W |  12860MiB / 15109MiB |     92%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [1]:
import numpy as np # linear algebra
import math
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import sys
from nltk.tokenize import TweetTokenizer
from emoji import demojize

import random
import math

from tqdm.notebook import tqdm
import string

from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
import tensorflow as tf
from tensorflow.keras import backend as K

import transformers
from transformers import TFAutoModel, TFRobertaModel, AutoTokenizer, TFAutoModelForSequenceClassification,TFAutoModelForSequenceClassification

from collections import Counter

print('Using Tensorflow version:', tf.__version__)
print('Using Transformers version:', transformers.__version__)

# import warnings
# warnings.filterwarnings('ignore')
tqdm.pandas()

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Using Tensorflow version: 2.3.0
Using Transformers version: 3.0.2


/home/leonard/leonard/ai_server_1_anaconda3/envs/tf230/lib/python3.7/site-packages/tqdm/std.py:701: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [2]:
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [3]:
seed = 1512
def seed_all(seed=1512):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    tf.random.set_seed(seed)
seed_all(seed)

In [4]:
# # %%time

# train_df = pd.read_csv("/home/leonard/leonard/nlp/ReINTEL/data/final_data/train_5_folds.csv")
# # test_df = pd.read_csv("/home/leonard/leonard/nlp/ReINTEL/data/final_data/test.csv")
# test_df = pd.read_csv("/home/leonard/leonard/nlp/ReINTEL/data/final_data/private_test.csv", index_col=0)


# train_df["post_message"] = train_df["post_message"].astype(str)
# test_df["post_message"] = test_df["post_message"].astype(str)

In [5]:
# train_df.head()

In [6]:
# test_df.head()

In [7]:
# roberta = '/home/leonard/leonard/nlp/ReINTEL/outputs/pretraining_vlsp' 
# roberta_tokenizer = AutoTokenizer.from_pretrained(roberta)
# # roberta_model = TFAutoModel.from_pretrained(roberta)
# roberta_model = TFRobertaModel.from_pretrained(roberta)

In [8]:
# train_len_word = [len(text.split()) for text in train_df.post_message]
# test_len_word = [len(text.split()) for text in test_df.post_message]
# test_len_char = [len(text) for text in train_df.post_message]
# test_len_char = [len(text) for text in test_df.post_message]

In [9]:
# def length_plot(lengths):
#     plt.figure(figsize=(12,7))
#     textstr = f' Mean: {np.mean(lengths):.2f} \u00B1 {np.std(lengths):.2f} \n Max: {np.max(lengths)}'
#     props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)

#     plt.text(0, 0, textstr, fontsize=14,
#             verticalalignment='top', bbox=props)
#     sns.countplot(lengths)    

In [10]:
# length_plot(train_len_word)

In [11]:
# length_plot(test_len_word)

In [12]:
# MAX_LEN = 256
# BATCH_SIZE = 48

In [13]:
# def regular_encode(texts, maxlen=MAX_LEN):

#     roberta_enc_di = roberta_tokenizer.batch_encode_plus(
#         texts,
#         return_token_type_ids=True,
#         pad_to_max_length=True,
#         max_length=maxlen,
#         truncation=True,
#     )
    
#     roberta_enc = (
#         np.array(roberta_enc_di["input_ids"]),
#         np.array(roberta_enc_di["attention_mask"]),
#         np.array(roberta_enc_di["token_type_ids"]),
#     )
#     return roberta_enc

In [14]:
# AUTO = tf.data.experimental.AUTOTUNE

# def data_generator(train_df, val_df):

#     X_train = regular_encode(train_df["post_message"].values, maxlen=MAX_LEN)
#     # y_train = tf.keras.utils.to_categorical(train_df['Label'].values, num_classes=2)
#     y_train = train_df["label"].values
#     X_val = regular_encode(val_df["post_message"].values, maxlen=MAX_LEN)
#     # y_val = tf.keras.utils.to_categorical(val_df['Label'].values, num_classes=2)
#     y_val = val_df["label"].values

#     train_dataset = (
#         tf.data.Dataset.from_tensor_slices((X_train, y_train))
#         .repeat()
#         .shuffle(1024)
#         .batch(BATCH_SIZE)
#         .prefetch(AUTO)
#     )

#     valid_dataset = (
#         tf.data.Dataset.from_tensor_slices((X_val, y_val))
#         .batch(BATCH_SIZE)
#         .cache()
#         .prefetch(AUTO)
#     )

#     return train_dataset, valid_dataset

In [15]:
# def build_model(bert_model_name_or_path, max_len=384, n_hiddens=-1):
#     bert_model = TFAutoModel.from_pretrained(bert_model_name_or_path)

#     bert_input_word_ids = tf.keras.layers.Input(
#         shape=(max_len,), dtype=tf.int32, name="bert_input_id"
#     )
#     bert_attention_mask = tf.keras.layers.Input(
#         shape=(max_len,), dtype=tf.int32, name="bert_attention_mask"
#     )
#     bert_token_type_ids = tf.keras.layers.Input(
#         shape=(max_len,), dtype=tf.int32, name="bert_token_type_ids"
#     )

#     bert_sequence_output = bert_model(
#         bert_input_word_ids,
#         attention_mask=bert_attention_mask,
#         token_type_ids=bert_token_type_ids,
# #         output_hidden_states=True,
# #         output_attentions=True,
        
#     )

#     # print(len(bert_sequence_output)) # 4

#     # print(bert_sequence_output[0].shape) # (None, max_len, 768)

#     # print(bert_sequence_output[1].shape) # (None, 768)
#     # print(len(bert_sequence_output[2])) # 13
#     # print(bert_sequence_output[2][0].shape) # (None, max_len, 768)
#     # print(len(bert_sequence_output[3])) # 12
#     # print(bert_sequence_output[3][0].shape) # (None, 12, None, max_len)

#     # TODO: get bert embedding

# #     if n_hiddens == -1:  # get [CLS] token embedding only
# #         # print("Get pooler output of shape (batch_size, hidden_size)")
# #         bert_sequence_output = bert_sequence_output[0][:, 0, :]
# #     else:  # concatenate n_hiddens final layer
# #         # print(f"Concatenate {n_hiddens} hidden_states of shape (batch_size, hidden_size)")
# #         bert_sequence_output = tf.concat(
# #             [bert_sequence_output[2][-i] for i in range(n_hiddens)], axis=-1)

#     # print("bert_sequence_output shape", bert_sequence_output.shape)

    
# # MLP    
# #     out = tf.keras.layers.Flatten()(bert_sequence_output)
# #     out = tf.keras.layers.Dense(1, activation="sigmoid")(out)
# # CNN
#     out = tf.keras.layers.Dropout(0.15)(bert_sequence_output[0])
#     out = tf.keras.layers.Conv1D(768, 2,padding='same')(out)
#     out = tf.keras.layers.LeakyReLU()(out)
#     out = tf.keras.layers.Conv1D(64, 2,padding='same')(out)
# #     out = tf.keras.layers.Dense(1)(out)
#     out = tf.keras.layers.Flatten()(out)
#     out = tf.keras.layers.Dense(1)(out)
#     out = tf.keras.layers.Activation('sigmoid')(out)
    
    
    
#     model = tf.keras.models.Model(
#         inputs=[
#             bert_input_word_ids,
#             bert_attention_mask,
#             bert_token_type_ids,  # bert input
#         ],
#         outputs=out,
#     )
#     model.compile(
#         tf.keras.optimizers.Adam(lr=5e-5),
#         loss="binary_crossentropy",
#         metrics=[tf.keras.metrics.AUC()],
#     )

#     return model

In [16]:
# %%time
# model = build_model(roberta, max_len=MAX_LEN)
# model.summary()

In [17]:
# n_splits = 5
# n_epochs = 5

# DISPLAY=1 # USE display=1 FOR INTERACTIVE
# exp = f'phobert_cnn_{MAX_LEN}_len'

# output_dir = f'../outputs/{exp}_models'
# os.makedirs(output_dir, exist_ok=True)

In [18]:
# def scheduler(epoch):
#     return 3e-5*0.2**epoch

In [19]:
# strategy = tf.distribute.MirroredStrategy()

In [20]:
# # for fold, (idxT, idxV) in enumerate(kf.split(train_df)):
# for fold in sorted(train_df["fold"].unique()):
# #     if fold<3:
# #         continue
#     print('*'*100)
#     print(f'FOLD: {fold+1}/{n_splits}')
    
#     K.clear_session()
#     with strategy.scope():
#         model = build_model(roberta, max_len=MAX_LEN)
# #     model = build_model(roberta, max_len=MAX_LEN)
        
#     reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

#     model_dir = os.path.join(output_dir, f'Fold_{fold+1}.h5')

#     sv = tf.keras.callbacks.ModelCheckpoint(model_dir, 
#                                             monitor='val_auc', 
#                                             verbose=1, 
#                                             save_best_only=True,
#                                             save_weights_only=True, 
#                                             mode='max', 
#                                             save_freq='epoch')
    
#     train_df_ = train_df[train_df["fold"]!=fold]
#     val_df_ = train_df[train_df["fold"]==fold]
#     train_dataset, valid_dataset = data_generator(train_df_, val_df_)
    
#     n_steps = train_df_.shape[0] // BATCH_SIZE + 1
#     train_history = model.fit(
#         train_dataset,
#         steps_per_epoch=n_steps,
        
#         callbacks=[sv, 
#             reduce_lr,
#             # tb
#             ],
#         validation_data=valid_dataset,
#         epochs=n_epochs
#     )

In [21]:
# X_test = regular_encode(test_df['post_message'].values, maxlen=MAX_LEN)
# y_test = np.zeros((len(test_df),1))
# test_dataset = (
#     tf.data.Dataset
#     .from_tensor_slices((X_test,y_test))
#     .batch(BATCH_SIZE)
# )

In [22]:
# model = build_model(roberta, max_len=MAX_LEN)
# preds = []
# for i, file_name in enumerate(os.listdir(output_dir)):
#     print('_'*80)
    
#     K.clear_session()
#     model_path = os.path.join(output_dir, file_name)
    
#     print(f'Inferencing with model from: {model_path}')
#     model.load_weights(model_path)

#     pred = model.predict(test_dataset,
#                          batch_size=128,
#                          verbose=DISPLAY)
#     # print(pred[])
#     preds.append(pred)

In [23]:
# preds = np.mean(preds, axis=0)
# test_df["prediction"] = preds
# test_df["prediction"].to_csv(f"{exp}.csv", header=False)

### EDA

In [27]:
# %%time

warmup_train_df = pd.read_excel("/home/leonard/leonard/my_work/ReINTEL/data/raw_data/warmup_training_dataset.xlsx", index_col="id")
warmup_test_df = pd.read_excel("/home/leonard/leonard/my_work/ReINTEL/data/raw_data/warmup_test_set.xlsx", index_col="id")

public_train_df = pd.read_csv("/home/leonard/leonard/my_work/ReINTEL/data/raw_data/public_train.csv")
raw_test_df = pd.read_csv("/home/leonard/leonard/my_work/ReINTEL/data/raw_data/public_test.csv")

# TODO: make use of warmup_test_df
raw_train_df = pd.concat([warmup_train_df, public_train_df]).drop_duplicates()

In [28]:
raw_train_df.head()

,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label,id
0,2167074723833126912,"Cần các bậc phụ huynh xã Ngũ Thái lên tiếng, k...",1.58443e+09,45,15,8,1,NaN
1,7368497547812410368,KÊU GỌI ĂN CHAY CẦU NGUYỆN XIN CHÚA CỨU KHỎI D...,1.58136e+09,979,39,138,1,NaN
2,-5754142625280308224,"Giàn khoan dầu khí gặp sự cố, chết người!\n\nG...",1.58704e+09,85,13,61,1,NaN
3,4177935308849652224,"Thuận Lợi có ca dương tính CV19 rồi đó, mọi ng...",1.59645e+09,114,12,5,1,NaN
4,5540309800745996288,Sa Pa cho 9 người khách nước ngoài đi cùng chu...,1.58355e+09,166,4,21,1,NaN


In [29]:
# %%time

train_df = pd.read_csv("/home/leonard/leonard/my_work/ReINTEL/data/final_data/train_5_folds.csv")
test_df = pd.read_csv("/home/leonard/leonard/my_work/ReINTEL/data/final_data/private_test.csv")
# test_df = pd.read_csv("/home/leonard/leonard/nlp/ReINTEL/data/final_data/test.csv")

train_df["post_message"] = train_df["post_message"].astype(str)
test_df["post_message"] = test_df["post_message"].astype(str)

In [30]:
def tokenized_text_normalize(text):
#     text=  re.sub(r'http(\S)+', ' ',text)
#     text=  re.sub(r'https(\S)+', ' ',text)
#     text=  re.sub(r'http ...', ' ',text)
#     text = re.sub(r'@[\S]+',' ',text)
#     text = text.strip(string.punctuation+" ")
#     text = re.sub("\*", " ", text)
#     text = re.sub("#", " ", text)
#     text = text.strip(r"# *" )
    text = re.sub(" _ ",  " ", text)
#     text = re.sub("\.+",  "\.", text)
    text = re.sub("…",  "...", text)
#     text = re.sub(r"https?://\S+|www\.\S+", "", text)
    text = re.sub(r"- - -", " ", text)
    text = re.sub("_ ", " ", text)
    text = re.sub(" +", " ", text)
    return text

In [31]:
# tokenized_text_normalize(input())

In [32]:
train_df["post_message"] = train_df["post_message"].apply(tokenized_text_normalize)
test_df["post_message"] = test_df["post_message"].apply(tokenized_text_normalize)

In [33]:
with open("train.txt", "w") as f:
    f.write("\n".join(train_df["post_message"].astype(str)))
with open("test.txt", "w") as f:
    f.write("\n".join(test_df["post_message"].astype(str)))

In [34]:
train_df.head()

,user_name,post_message,timestamp_post,num_like_post,num_comment_post,num_share_post,label,id,fold
0,2.16707472383313E+018,"Cần các bậc phụ_huynh xã Ngũ_Thái lên_tiếng , ...",1.584426e+09,45,15,8,1,NaN,3
1,7.36849754781241E+018,KÊU_GỌI ĂN_CHAY CẦU_NGUYỆN XIN CHÚA CỨU KHỎI D...,1.581363e+09,979,39,138,1,NaN,1
2,-5.75414262528031E+018,"Giàn khoan dầu_khí gặp sự_cố , chết người ! Gi...",1.587042e+09,85,13,61,1,NaN,3
3,4.17793530884965E+018,"Thuận_Lợi có ca dương_tính CV19 rồi đó , mọi n...",1.596449e+09,114,12,5,1,NaN,1
4,5.540309800746E+018,Sa_Pa cho 9 người khách nước_ngoài đi cùng chu...,1.583548e+09,166,4,21,1,NaN,3


In [35]:
def extract_num(text):
    if type(text)==str:
        if text=="unknown":
            return math.nan
        return re.findall("\d+", text)[0]
    return text

In [36]:
for col in ["timestamp_post", "num_like_post", "num_comment_post", "num_share_post"]:
    train_df[col] = train_df[col].apply(extract_num)
    train_df[col] = train_df[col].astype(float)
    train_df[col] = train_df[col].fillna(-train_df[col].mean())
    test_df[col] = test_df[col].apply(extract_num)
    test_df[col] = test_df[col].astype(float)
    test_df[col] = test_df[col].fillna(-test_df[col].mean())    

In [37]:
# nan_dict = {"timestamp_post": 1e10, "num_like_post":-2,"num_comment_post":-2, "num_share_post":-2}
# unknown_dict = {"timestamp_post": 1e10, "num_like_post":-2,"num_comment_post":-2, "num_share_post":-2}


# def vlsp_impute(text, field):
#     if type(text)!=str:
#         if math.isnan(text):
#             return nan_dict[field]
#     elif text=="unknown":        
#         return unknown_dict[field]
#     else:
#         try:
#             return int(extract_num(text))
#         except:
#             print(text)
#             return nan_dict[field]

In [38]:
# scaler = StandardScaler()
# scaler.fit(pd.concat([train_df, test_df])[["timestamp_post", "num_like_post", "num_comment_post", "num_share_post"]].values)
# train_df[["timestamp_post", "num_like_post", "num_comment_post", "num_share_post"]] = scaler.transform(train_df[["timestamp_post", "num_like_post", "num_comment_post", "num_share_post"]])
# test_df[["timestamp_post", "num_like_post", "num_comment_post", "num_share_post"]] = scaler.transform(test_df[["timestamp_post", "num_like_post", "num_comment_post", "num_share_post"]])

In [39]:
# train_df["timestamp_post"] = train_df["timestamp_post"].fillna(-1e10)
# test_df["timestamp_post"] = test_df["timestamp_post"].fillna(-1e10)

In [43]:
MAX_LEN = 256
BATCH_SIZE = 48
roberta = 'vinai/phobert-base' 
roberta_tokenizer = AutoTokenizer.from_pretrained("/home/leonard/leonard/my_work/ReINTEL/pretrained_phobert-base", use_fast=False)

OSError: Model name '/home/leonard/leonard/my_work/ReINTEL/pretrained_phobert-base' was not found in tokenizers model name list (roberta-base, roberta-large, roberta-large-mnli, distilroberta-base, roberta-base-openai-detector, roberta-large-openai-detector). We assumed '/home/leonard/leonard/my_work/ReINTEL/pretrained_phobert-base' was a path, a model identifier, or url to a directory containing vocabulary files named ['vocab.json', 'merges.txt'] but couldn't find such vocabulary files at this path or url.

In [39]:
def regular_encode(df, max_len=MAX_LEN):
    
    roberta_enc_di = roberta_tokenizer.batch_encode_plus(
        df["post_message"].values,
        return_token_type_ids=True,
        pad_to_max_length=True,
        max_length=max_len,
        truncation=True,
    )
    timestamp_post = (df["timestamp_post"]/1e13).values
    num_like_post = (df["num_like_post"]/1e8).values
    num_comment_post = (df["num_comment_post"]/1e8).values
    num_share_post = (df["num_share_post"]/1e8).values
    
    roberta_enc = (
        np.array(roberta_enc_di["input_ids"]),
        np.array(roberta_enc_di["attention_mask"]),
        np.array(roberta_enc_di["token_type_ids"]),
        timestamp_post,
        num_like_post,
        num_comment_post,
        num_share_post,
#         np.concatenate([num_like_post, num_comment_post, num_share_post], axis=0)
    )
    
    
    return roberta_enc

In [40]:
AUTO = tf.data.experimental.AUTOTUNE

def data_generator(train_df, val_df):

    X_train = regular_encode(train_df, max_len=MAX_LEN)
    # y_train = tf.keras.utils.to_categorical(train_df['Label'].values, num_classes=2)
    y_train = train_df["label"].values
    X_val = regular_encode(val_df, max_len=MAX_LEN)
    # y_val = tf.keras.utils.to_categorical(val_df['Label'].values, num_classes=2)
    y_val = val_df["label"].values

    train_dataset = (
        tf.data.Dataset.from_tensor_slices((X_train, y_train))
        .repeat()
        .shuffle(1024)
        .batch(BATCH_SIZE)
        .prefetch(AUTO)
    )

    valid_dataset = (
        tf.data.Dataset.from_tensor_slices((X_val, y_val))
        .batch(BATCH_SIZE)
        .cache()
        .prefetch(AUTO)
    )

    return train_dataset, valid_dataset

In [41]:
def build_model(bert_model_name_or_path="vinai/phobert-base", max_len=384, n_hiddens=-1):
    bert_model = TFAutoModel.from_pretrained(bert_model_name_or_path)

    bert_input_word_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="bert_input_id"
    )
    bert_attention_mask = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="bert_attention_mask"
    )
    bert_token_type_ids = tf.keras.layers.Input(
        shape=(max_len,), dtype=tf.int32, name="bert_token_type_ids"
    )

    bert_sequence_output = bert_model(
        bert_input_word_ids,
        attention_mask=bert_attention_mask,
        token_type_ids=bert_token_type_ids,
        output_hidden_states=True,
        output_attentions=True,
        
    )

    # print(len(bert_sequence_output)) # 4

    # print(bert_sequence_output[0].shape) # (None, max_len, 768)

    # print(bert_sequence_output[1].shape) # (None, 768)
    # print(len(bert_sequence_output[2])) # 13
    # print(bert_sequence_output[2][0].shape) # (None, max_len, 768)
    # print(len(bert_sequence_output[3])) # 12
    # print(bert_sequence_output[3][0].shape) # (None, 12, None, max_len)

    # TODO: get bert embedding

    if n_hiddens == -1:  # get [CLS] token embedding only
        # print("Get pooler output of shape (batch_size, hidden_size)")
        bert_sequence_output = bert_sequence_output[0][:, 0, :]
#         bert_sequence_output = bert_sequence_output[1]
    else:  # concatenate n_hiddens final layer
        # print(f"Concatenate {n_hiddens} hidden_states of shape (batch_size, hidden_size)")
        bert_sequence_output = tf.concat(
            [bert_sequence_output[2][-i] for i in range(n_hiddens)], axis=-1)
        bert_sequence_output = bert_sequence_output[:, 0, :]

    # print("bert_sequence_output shape", bert_sequence_output.shape)

#     bert_output = tf.keras.layers.Flatten()(bert_sequence_output)
    bert_output = tf.keras.layers.Dense(8, activation="relu")(bert_sequence_output)
#     print(bert_output.shape)
    
    timestamp_post = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name="timestamp_post")
    num_like_post = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name="num_like_post")
    num_comment_post = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name="num_comment_post")
    num_share_post = tf.keras.layers.Input(shape=(1,), dtype=tf.float32, name="num_share_post")
    
    aulixiary_info = tf.keras.layers.Concatenate()([timestamp_post, num_like_post, num_comment_post, num_share_post])
#     aulixiary_output  = tf.keras.layers.GaussianNoise(0.2)(aulixiary_info)
    
    out = tf.keras.layers.Concatenate()([bert_output, aulixiary_info]) 
#     print(out.shape)
    out = tf.keras.layers.Dense(1, activation="sigmoid")(out)

    model = tf.keras.models.Model(
        inputs=[
            bert_input_word_ids,
            bert_attention_mask,
            bert_token_type_ids,  # bert input
            timestamp_post,
            num_like_post,
            num_comment_post,
            num_share_post,
        ],
        outputs=out,
    )
    model.compile(
        tf.keras.optimizers.Adam(lr=5e-5),
        loss="binary_crossentropy",
        metrics=[tf.keras.metrics.AUC()],
    )

    return model

In [42]:
%%time
model = build_model(max_len=MAX_LEN)
model.summary()

Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.


Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
bert_input_id (InputLayer)      [(None, 256)]        0                                            
__________________________________________________________________________________________________
bert_attention_mask (InputLayer [(None, 256)]        0                                            
__________________________________________________________________________________________________
bert_token_type_ids (InputLayer [(None, 256)]        0                                            
__________________________________________________________________________________________________
tf_roberta_model (TFRobertaMode ((None, 256, 768), ( 134998272   bert_input_id[0][0]              
                                                                 bert_attention_mask[0]

In [43]:
n_splits = 5
n_epochs = 5

DISPLAY=1 # USE display=1 FOR INTERACTIVE
exp = f'phobert+auxiliary_{MAX_LEN}_len'

output_dir = f'../outputs/{exp}_models'
os.makedirs(output_dir, exist_ok=True)

In [44]:
def scheduler(epoch):
    return 3e-5*0.2**epoch

In [45]:
strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


In [ ]:
# for fold, (idxT, idxV) in enumerate(kf.split(train_df)):
for fold in sorted(train_df["fold"].unique()):
    print('*'*100)
    print(f'FOLD: {fold+1}/{n_splits}')
    if fold<4:
        continue
    K.clear_session()
    with strategy.scope():
        model = build_model(max_len=MAX_LEN)
        
    reduce_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)

    model_dir = os.path.join(output_dir, f'Fold_{fold+1}.h5')

    sv = tf.keras.callbacks.ModelCheckpoint(model_dir, 
                                            monitor='val_auc', 
                                            verbose=1, 
                                            save_best_only=True,
                                            save_weights_only=True, 
                                            mode='max', 
                                            save_freq='epoch')
    
    train_df_ = train_df[train_df["fold"]!=fold]
    val_df_ = train_df[train_df["fold"]==fold]
    train_dataset, valid_dataset = data_generator(train_df_, val_df_)
    
    n_steps = train_df_.shape[0] // BATCH_SIZE + 1
    train_history = model.fit(
        train_dataset,
        steps_per_epoch=n_steps,
        
        callbacks=[sv, 
            reduce_lr,
            # tb
            ],
        validation_data=valid_dataset,
        epochs=n_epochs
    )

****************************************************************************************************
FOLD: 1/5
****************************************************************************************************
FOLD: 2/5
****************************************************************************************************
FOLD: 3/5
****************************************************************************************************
FOLD: 4/5
****************************************************************************************************
FOLD: 5/5


Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
/home/leonard/leonard/ai_server_1_anaconda3/envs/tf230/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2022: FutureWarning: The `pad_to_max_l

Epoch 1/5
Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:batch_all_reduce: 198 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task

In [ ]:
# 0.94458 0.94645 0.95718 0.94500

In [ ]:
X_test = regular_encode(test_df, max_len=MAX_LEN)
y_test = np.zeros((len(test_df),1))
test_dataset = (
    tf.data.Dataset
    .from_tensor_slices((X_test,y_test))
    .batch(BATCH_SIZE)
)

In [ ]:
model = build_model(max_len=MAX_LEN)
preds = []
for i, file_name in enumerate(os.listdir(output_dir)):
    print('_'*80)
    
    K.clear_session()
    model_path = os.path.join(output_dir, file_name)
    
    print(f'Inferencing with model from: {model_path}')
    model.load_weights(model_path)

    pred = model.predict(test_dataset,
                         batch_size=128,
                         verbose=DISPLAY)
    # print(pred[])
    preds.append(pred)

In [ ]:
preds = np.mean(preds, axis=0)
test_df["prediction"] = preds
test_df["prediction"].to_csv(f"{exp}.csv", header=False)

## pretraining

In [ ]:
import pandas as pd
import re
from data.tokenizer import VnCoreTokenizer
from tqdm import tqdm
tqdm.pandas()
vncoretokenizer = VnCoreTokenizer()

In [ ]:
def text_normalize(text):
    text = text.strip()
    text = re.sub("^TTO *- *", "", text)    
    text = re.sub("^VOV\.VN *- *", "", text)
#     text = vncoretokenizer.tokenize(text)
    return text

In [ ]:
news_df = pd.read_csv("/home/leonard/leonard/nlp/ReINTEL/data/newdata_21112020.csv")[["title", "content"]].dropna()

In [ ]:
news_df["content"] = news_df["content"].apply(text_normalize)

In [ ]:
news_df["title"] = news_df["title"].apply(text_normalize)

In [ ]:
news_df.head(20)

In [ ]:
# news_df.to_csv("/home/leonard/leonard/nlp/ReINTEL/data/tokenized_news_data.csv", index=False)

In [ ]:
# news_df = pd.read_csv("/home/leonard/leonard/nlp/ReINTEL/data/tokenized_news_data.csv")
# news_df.head()

In [ ]:
all_documents = []

for index, row in tqdm(news_df.iterrows(), total=len(news_df)):
    title = vncoretokenizer.tokenize(str(row["title"]))
    text = str(row["content"])
    sentences = vncoretokenizer.tokenize(text, return_sentences=True)
    document = [title] + sentences
    all_documents.append(document)

In [ ]:
import random
random.shuffle(all_documents)

In [ ]:
train_documents = all_documents[:int(len(all_documents)*0.8)]
val_documents = all_documents[int(len(all_documents)*0.8):]
print(len(train_documents), len(val_documents))

In [ ]:
# with open("/home/leonard/leonard/nlp/ReINTEL/data/tokenized_news_train.txt", "w") as f:
#     for document in tqdm(train_documents):
#         for sentence in document:
#             f.write(sentence)
#             f.write("\n")
#         f.write("\n")

In [ ]:
# with open("/home/leonard/leonard/nlp/ReINTEL/data/tokenized_news_val.txt", "w") as f:
#     for document in tqdm(val_documents):
#         for sentence in document:
#             f.write(sentence)
#             f.write("\n")
#         f.write("\n")

In [ ]:
warmup_train_df = pd.read_excel("/home/leonard/leonard/nlp/ReINTEL/data/raw_data/warmup_training_dataset.xlsx", index_col="id")
warmup_test_df = pd.read_excel("/home/leonard/leonard/nlp/ReINTEL/data/raw_data/warmup_test_set.xlsx", index_col="id")

public_train_df = pd.read_csv("/home/leonard/leonard/nlp/ReINTEL/data/raw_data/public_train.csv")
test_df = pd.read_csv("/home/leonard/leonard/nlp/ReINTEL/data/raw_data/public_test.csv")

# TODO: make use of warmup_test_df
train_df = pd.concat([warmup_train_df, public_train_df]).drop_duplicates()

In [ ]:
train_documents = train_df["post_message"].values
test_documents = test_df["post_message"].values

In [ ]:
train_all_documents = []

for index, row in tqdm(train_df.iterrows(), total=len(train_df)):
    text = str(row["post_message"])
    sentences = vncoretokenizer.tokenize(text, return_sentences=True)
    train_all_documents.append(sentences)

In [ ]:
import random
random.shuffle(train_all_documents)

In [ ]:
test_all_documents = []

for index, row in tqdm(test_df.iterrows(), total=len(test_df)):
    text = str(row["post_message"])
    sentences = vncoretokenizer.tokenize(text, return_sentences=True)
    test_all_documents.append(sentences)
    

In [ ]:
total = len(train_all_documents) + len(test_all_documents )
test_size = int(total*0.2)
train_vlsp_text = test_all_documents + train_all_documents[:-test_size]
val_vlsp_text = train_all_documents[-test_size:]
print(len(train_vlsp_text), len(val_vlsp_text))

In [ ]:
with open("/home/leonard/leonard/nlp/ReINTEL/data/tokenized_train_vlsp_text.txt", "w") as f:
    for document in tqdm(train_vlsp_text):
        for sentence in document:
            f.write(sentence)
            f.write("\n")
        f.write("\n")

In [ ]:
with open("/home/leonard/leonard/nlp/ReINTEL/data/tokenized_val_vlsp_text.txt", "w") as f:
    for document in tqdm(val_vlsp_text):
        for sentence in document:
            f.write(sentence)
            f.write("\n")
        f.write("\n")

## RobertaForMaskedLM

In [ ]:
import transformers
from transformers import TFAutoModel, AutoTokenizer, TFAutoModelForMaskedLM
import tensorflow as tf

In [ ]:
MAX_LEN = 128
BATCH_SIZE = 16

In [ ]:
pretrain_model = 'vinai/phobert-base'
tokenizer =  AutoTokenizer.from_pretrained(pretrain_model)
model = TFAutoModelForMaskedLM.from_pretrained(pretrain_model)

In [ ]:
def build_model():
    print(f'Using pretrained {pretrain_model}')
    model = TFAutoModelForMaskedLM.from_pretrained(pretrain_model)
    optimizer = tf.keras.optimizers.Adam(lr=5e-5)
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    model.compile(optimizer, loss=loss, metrics=["accuracy"])
    return model

In [ ]:
# model = build_model()
# model.summary()

In [ ]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="/home/leonard/leonard/nlp/ReINTEL/data/tokenized_news_train.txt",
    block_size=128,
)

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/home/leonard/leonard/nlp/ReINTEL/outputs",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_gpu_train_batch_size=64,
    save_steps=10_000,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

In [ ]:
## post process

In [ ]:
best_prediction = pd.read_csv("/home/leonard/leonard/nlp/ReINTEL/vlsp_reintel_text/results.csv", header=None)
test_df = pd.read_csv("/home/leonard/leonard/nlp/ReINTEL/data/raw_data/public_test.csv")

In [ ]:
best_prediction.head()

In [ ]:
test_df["prediction"] = best_prediction[1].values

In [ ]:
test_df.head()

In [ ]:
def cutoff(prob):
    if prob>0.92:
        return 1.0
    return prob

In [ ]:
# for index, row in test_df.iterrows():
#     if "url" in str(row["post_message"]).lower() and len(str(row["post_message"])) < 20:
#         print("*"*50)
#         print(row["post_message"])
#         print(test_df.loc[index, "prediction"])
# #         test_df.loc[index, "prediction"] = 0.005
# #         print(test_df.loc[index, "prediction"])

In [ ]:
test_df["prediction"].plot()

In [ ]:
test_df["prediction"].apply(cutoff).to_csv(f"post_process.csv", header=False)

In [ ]:
test_df["prediction"].apply(cutoff).plot()